In [1]:
from lusidtools.jupyter_tools import toggle_code

"""FX Inference Triangulation

Attributes
----------
FX
Foreign Exchange
FX inference
Triangulation
"""

toggle_code("Toggle Docstring")

# FX Inference Methodology - Triangulation

In this notebook we demonstrate the FX inference methodology in LUSID. When the market option `attempt_to_infer_missing_fx=True` is used, LUSID will attempt to infer FX quotes from the available market data if no direct quote is found according to a set of rules which are described below and will be demonstrated in this notebook.

In attempting to infer an FX rate for a currency pair XXXYYY (e.g. XXX=AUD and YYY=JPY for AUDJPY), LUSID performs the following steps, stopping the process and returning an inferred quote as soon as one is found:

- Check if `XXXYYY` is present. If so, return `XXXYYY`.
- Check if `YYYXXX` is present. If so, return `1 / YYYXXX`.
- Attempt to triangulate through a base currency (USD, GBP, EUR) in order:
    - Triangulate through USD: Given at least one of USDXXX and XXXUSD and at least one of USDYYY and YYYUSD, LUSID will calculate an XXXYYY rate (See [section 2.3](#23-currency-pair-quoting-conventions) for exact ordering).
    - Triangulate through GBP: as above.
    - Triangulate through EUR: as above.
- Otherwise, this is an error state. Insufficient market data.

The list of base currencies used for triangulation is hardcoded in LUSID as (USD, GBP, EUR) and is not configurable. Triangulation will be attempted first through USD, then GBP, then EUR. 

The `effective_at` date is not taken into consideration when attempting to infer FX rates, provided the quotes lie in the quote interval specified in the market data key rules. That is to say, the first valid quote will be derived according the FX inference rules and priorities described in this notebook, without attempting to rank derived quotes using the effective dates.

**Example:** Suppose we are running a valuation on Friday 13 September 2024 which requires a EURJPY quote and the market data store contains a EURJPY quote `q1 = 157.0130` effective on Thu 12/09/24 and a JPYEUR quote `q2 = 1/156.75 = 0.00637959` effective on Fri 13/09/24. We could either use the direct EURJPY quote `q1`, which is one day old, or infer a EURJPY quote `1/q2 = 156.75` effective at the valuation date. In LUSID, the direct quote will be used if available without considering derived quotes formed from one or more newer FX rate quotes.

When triangulating through one of the base currencies (USD, GBP, EUR), the latest quote for each currency pair will be used rather than attempting to match quotes on the same date.

**Example:** Suppose we are running a valuation on Friday 13 September 2024 that requires a EURJPY quote and the market date store contains the following FX quotes:

| Currency Pair | Rate   | Effective At |
| ------------- | ----   | ------------ |
| EURUSD        | 1.1011 | 12/09/2024   |
| USDJPY        | 142.59 | 12/09/2024   |
| EURUSD        | 1.1078 | 13/09/2024   |

Combining the EURUSD and USDJPY quotes effective on Thursday 12 September 2024 gives an inferred quote `EURJPY = 1.1011 * 142.59 = 157.005849`. If instead we combine the latest quote for each currency pair, as LUSID does, we derive a quote `EURJPY = 1.1078 * 142.59 = 157.9612`. It is recommended to upsert all market data effective on the valuation date wherever possible.

The triangulation methodology only uses a single value for each currency pair, without taking bid/ask prices into account.

The FX inference methodology is the same whether we are dealing with single FX quotes or curves (for curves, we multiply, divide and scale pointwise), so to focus on the inference rules we will only consider single quotes and ignore complex market data.

## Table of Contents

1. [Setup](#1-setup)
    1. [Portfolio Creation](#11-portfolio-creation)
    2. [Create Holdings](#12-create-holdings)
    3. [Pricing Recipes and Market Data](#13-pricing-recipes-and-market-data)
    4. [Valuation](#14-valuation)
2. [Examples](#2-examples)
    1. [Inverse quote](#21-inverse-quote)
    2. [Triangulation through a base currency](#22-triangulation-through-a-base-currency)
    3. [Currency pair quoting conventions](#23-currency-pair-quoting-conventions)
    4. [Multi-step triangulation](#24-multi-step-triangulation)
    5. [Minor currencies](#25-minor-currencies)



## 1. Setup

In [2]:
# Import generic non-LUSID packages
import os
import pandas as pd
from datetime import datetime, timedelta
import pytz
from IPython.core.display import HTML
import json

# Import key modules from the LUSID package
import lusid
import lusid.models as lm
from lusid.utilities import ApiClientFactory

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.4f}".format
display(HTML("<style>.container { width: 90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate user and create API client
api_factory = ApiClientFactory(api_secrets_filename=secrets_path)
build_version = (
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version
)
display("LUSID environment initialised")
display(f"LUSID API version: {build_version}")

'LUSID environment initialised'

'LUSID API version: 0.6.13544.0'

In [3]:
# Set required APIs
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
aggregation_api = api_factory.build(lusid.api.AggregationApi)

### 1.1 Portfolio Creation

We now create and upsert a portfolio denominated in CHF with creation date Wednesday 01 May 2024.

In [4]:
# Define common variables
pf_scope = "fx-inference-triangulation"
pf_code = "FxInferenceTriangulation"
pf_ccy = "CHF"
pf_created_at = datetime(2024, 5, 1, tzinfo=pytz.utc)  # Wed 01 May

In [5]:
# Get or create portfolio
try:
    portfolio_api.get_portfolio(scope=pf_scope, code=pf_code)
    display(f"Found portfolio {pf_code} in scope {pf_scope}")
except lusid.ApiException as e:
    body = json.loads(e.body)
    if body["name"] == "PortfolioNotFound":
        transaction_portfolios_api.create_portfolio(
            scope=pf_scope,
            create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
                display_name=pf_code,
                code=pf_code,
                base_currency=pf_ccy,
                created=pf_created_at.isoformat(),
            ),
        )

        display(f"Created new portfolio {pf_code} in scope {pf_scope}")

    else:
        display(
            f"Error fetching portfolio - Title: {body['title']}, Details: {body['detail']}, Errors: {body['errors']}"
        )

'Found portfolio FxInferenceTriangulation in scope fx-inference-triangulation'

### 1.2 Create Holdings

We now create and upsert an `Equity` instrument denominated in AUD.

In [6]:
equity_id = "AUS12321"
equity_name = "AnAustralianCompany"
equity_ccy = "AUD"

equity = lm.Equity(
    identifiers={"ClientInternal": equity_id},
    dom_ccy=equity_ccy,
    instrument_type=lm.InstrumentType.EQUITY,
)

equity_definition = lm.InstrumentDefinition(
    name=equity_name,
    identifiers={"ClientInternal": lm.InstrumentIdValue(value=equity_id)},
    definition=equity,
)

In [7]:
try:
    upsert_response = instruments_api.upsert_instruments(
        request_body={equity_id: equity_definition}
    )
    luid = upsert_response.values[equity_id].lusid_instrument_id
    display(f"Success! Upserted instrument {luid}")
except KeyError as e:
    display(
        f"Failed to upsert instrument {equity_id}. Details: {upsert_response.failed[equity_id].detail}"
    )
except lusid.ApiException as e:
    body = json.loads(e.body)
    display(
        f"Title: {body['title']}, Details: {body['detail']}, Errors: {body['errors']}"
    )

'Success! Upserted instrument LUID_000185GZ'

Having created a transaction portfolio and an `Equity` instrument, we now add a `StockIn` transaction (Monday 03 June 2024) to create a long position of 10,000 units of the equity at 100 AUD each, rather than using  a `Buy/Sell` transaction which would affect the cash holding of the portfolio.

In [8]:
# Trade variables
trade_date = datetime(2024, 6, 3, tzinfo=pytz.utc)  # Monday 03 June 2024
settle_days = 2
settlement_date = trade_date + timedelta(days=settle_days)
units = 10_000
unit_price = 100  # AUD

total_consideration = units * unit_price

# Create StockIn transaction
stock_in_txn = lm.TransactionRequest(
    transaction_id="TXN001",
    type="StockIn",
    instrument_identifiers={"Instrument/default/ClientInternal": equity_id},
    transaction_date=trade_date.isoformat(),
    settlement_date=settlement_date.isoformat(),
    units=units,
    transaction_price=lm.TransactionPrice(price=unit_price, type="Price"),
    transaction_currency=equity_ccy,
    total_consideration=lm.CurrencyAndAmount(
        amount=total_consideration, currency=equity_ccy
    ),
    exchange_rate=1,
)

# Upsert StockIn transaction
try:
    response = transaction_portfolios_api.upsert_transactions(
        scope=pf_scope, code=pf_code, transaction_request=[stock_in_txn]
    )
    display(f"Transaction successfully updated at time {response.version.as_at_date}")

except lusid.ApiException as e:
    body = json.loads(e.body)
    display(
        f"Failed to upsert transaction - Title: {body['title']}, Details: {body['detail']}, Errors: {body['errors']}"
    )

'Transaction successfully updated at time 2024-09-13 12:53:11.031480+00:00'

### 1.3 Pricing Recipes and Market Data

Now we create some helper functions for upserting pricing recipes and quotes. FX inference is enabled in the market context setting `attempt_to_infer_missing_fx=True` in `MarketOptions`. We will use the `SimpleStatic` pricing model to value the `Equity`, which gives a valuation (in the equity base currency) based on a single price per unit quote for the equity, multiplying by the number of units held. The valuation in the portfolio base currency is given by converting at the spot rate. In our example, the portfolio currency is CHF and the equity is quoted in AUD, so the PV in CHF is given by multiplying the PV in AUD by a AUDCHF spot rate.

In [9]:
market_supplier = "Lusid"


def create_market_context(scope: str) -> lm.MarketContext:
    return lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Quote.ClientInternal.*",
                data_scope=scope,
                supplier=market_supplier,
                quote_type="Price",
                field="mid",
                quote_interval="1D",
            ),
            lm.MarketDataKeyRule(
                key="Fx.*.*",
                data_scope=scope,
                supplier=market_supplier,
                quote_type="Rate",
                field="mid",
                quote_interval="5D",
            ),
        ],
        options=lm.MarketOptions(
            default_supplier=market_supplier,
            default_scope=scope,
            default_instrument_code_type="ClientInternal",
            attempt_to_infer_missing_fx=True,
        ),
    )


def create_pricing_context() -> lm.PricingContext:
    return lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier=market_supplier,
                model_name="SimpleStatic",
                instrument_type="Equity",
            ),
        ],
    )


def upsert_recipe(scope: str, code: str):
    market_context = create_market_context(scope)
    pricing_context = create_pricing_context()
    description = "SimpleStatic pricing: " + scope

    pricing_recipe = lm.ConfigurationRecipe(
        scope=scope,
        code=code,
        description=description,
        market=market_context,
        pricing=pricing_context,
    )

    recipe_request = lm.UpsertRecipeRequest(configuration_recipe=pricing_recipe)
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=recipe_request
    )

Now we create helper functions for creating and upserting quotes.

In [10]:
def create_fx_quote(
    fx_pair: str, effective_at: datetime, rate: float
) -> lm.UpsertQuoteRequest:
    return lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                provider="Lusid",
                instrument_id=fx_pair,
                instrument_id_type="CurrencyPair",
                quote_type="Rate",
                field="mid",
            ),
            effective_at=effective_at.isoformat(),
        ),
        metric_value=lm.MetricValue(value=rate, unit="rate"),
        lineage="InternalSystem",
    )


def upsert_quotes(scope: str, quotes: list[lm.UpsertQuoteRequest]):
    # Create request body object from list of quotes
    request_body = {f"{i}": x for i, x in zip(range(len(quotes)), quotes)}
    quote_response = quotes_api.upsert_quotes(scope=scope, request_body=request_body)

    if quote_response.failed == {}:
        display(
            f"Quotes successfully loaded into LUSID. {len(quote_response.values)} quotes upserted."
        )
    else:
        display(
            f"Some failures occurred during quote upsert. Failed: {quote_response.failed}"
        )

### 1.4 Valuation

We will value the portfolio on Friday 13 September 2024.

In [11]:
valuation_date = datetime(2024, 9, 13, tzinfo=pytz.utc)  # Friday 13 September 2024

Now create a helper function to perform the valuation and return the results in a table.

In [12]:
def get_daily_valuation(
    market_data_scope: str,
    date: datetime,
    recipe_code: str,
    report_ccy: str = pf_ccy,
) -> pd.DataFrame:
    metrics = [
        lm.AggregateSpec(key="Instrument/CoreData/Name", op="Value"),
        lm.AggregateSpec(key="Valuation/PV", op="Value"),
        lm.AggregateSpec(key="Valuation/PV/Ccy", op="Value"),
        lm.AggregateSpec(key="Valuation/PvInPortfolioCcy", op="Value"),
    ]

    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=market_data_scope, code=recipe_code),
        metrics=metrics,
        group_by=None,
        portfolio_entity_ids=[lm.PortfolioEntityId(scope=pf_scope, code=pf_code)],
        valuation_schedule=lm.ValuationSchedule(effective_at=date.isoformat()),
        report_currency=report_ccy,
    )

    valuation_data = aggregation_api.get_valuation(
        valuation_request=valuation_request
    ).data

    valuation_df = pd.DataFrame(valuation_data)

    # Reorder columns:
    valuation_df = valuation_df.loc[
        :,
        [
            "Instrument/CoreData/Name",
            "Valuation/PV",
            "Valuation/PV/Ccy",
            "Valuation/PvInPortfolioCcy",
        ],
    ]

    return valuation_df

## 2. Examples

### 2.1 Inverse quote

When resolving the AUDCHF quotes, direct AUDCHF quotes will always take precedence over quotes inferred from an inverse rate CHFAUD or triangulated through a third currency, even if a quote could be inferred from more recent market data than the direct quote. In particular, if there is an AUDCHF quote effective the day before the valuation date (Thu 12 September 2024) and a CHFAUD quote effective on the valuation date (Fri 13 September 2024), then the AUDCHF quote will be used, rather than `1 / CHFAUD`.

We will use the following FX rates:

| FX Pair | Effective At   | Rate       |
| ------- | ------------   | ---------- |
| AUDCHF  | Thu 12/09/2024 | 0.5720     |
| CHFAUD  | Fri 13/09/2024 | 1 / 0.5671 |

In [13]:
market_data_scope = "fx-inference-inverse-quote-only"
recipe_code = "fx-inference-inverse-quote-only"
upsert_recipe(scope=market_data_scope, code=recipe_code)

We now upsert a quote of 100 AUD per unit of the equity, which will make the inferred FX rates easily readable from the valuation results.

In [14]:
# Upsert a quote for the equity. Assume the equity price has not changed for clarity.
equity_price = 100

quote_equity = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id=equity_id,
            instrument_id_type="ClientInternal",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=equity_price, unit=equity_ccy),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_equity])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

In order to value the portfolio in the portfolio currency, we need to convert the value of the holding from AUD to CHF by multiplying by the AUDCHF spot rate effective on the valuation date. First, we will only upsert a quote for CHFAUD, effective on the valuation date, so this rate will be inverted to infer an AUDCHF quote.

In [15]:
spot_chf_aud = 1 / 0.5671
effective_at = valuation_date
quote_chf_aud = create_fx_quote(
    fx_pair="CHF/AUD", effective_at=effective_at, rate=spot_chf_aud
)

upsert_quotes(scope=market_data_scope, quotes=[quote_chf_aud])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

With a holding of `10,000` units, purchased at `100 AUD` each, which are now still worth `100 AUD` each, the present value of the holding is `1,000,000 AUD`. Given a CHFAUD rate of `1 / 0.5671`, it is inverted to give a derived AUDCHF quote of `0.5671`. The PV of the holding in the portfolio currency CHF is given by multiplying by the AUDCHF spot rate, so is `0.5671 * 1,000,000 = 567,100.00 CHF`.

In [16]:
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope, date=valuation_date, recipe_code=recipe_code
)

display(valuation_df)

,Instrument/CoreData/Name,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInPortfolioCcy
0,AnAustralianCompany,"1,000,000.0000",AUD,"567,100.0000"


Now upsert the direct quote, effective on the day before the valuation date, along with the equity quote and the CHFAUD quote as above. We will see that the direct (but older) quote is used, rather than using the more recent inverse quote.

In [17]:
market_data_scope = "fx-inference-inverse-quote-vs-direct"
recipe_code = "fx-inference-inverse-quote-vs-direct"
upsert_recipe(scope=market_data_scope, code=recipe_code)

In [18]:
spot_aud_chf = 0.5720
effective_at = valuation_date + timedelta(days=-1)
quote_aud_chf = create_fx_quote(
    fx_pair="AUD/CHF", effective_at=effective_at, rate=spot_aud_chf
)

upsert_quotes(
    scope=market_data_scope, quotes=[quote_equity, quote_chf_aud, quote_aud_chf]
)

'Quotes successfully loaded into LUSID. 3 quotes upserted.'

With a holding of `10,000` units of the equity, each worth `100 AUD`, the present value of the holding is `1,000,000 AUD`, which is then converted to the portfolio currency CHF by multiplying by an AUDCHF spot rate `S`. The spot rate taken from the direct quote `S=0.5720` gives the PV in portfolio currency as `1,000,000 * 0.5720 = 572,000.00 CHF`.

In [19]:
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope, date=valuation_date, recipe_code=recipe_code
)

display(valuation_df)

,Instrument/CoreData/Name,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInPortfolioCcy
0,AnAustralianCompany,"1,000,000.0000",AUD,"572,000.0000"


### 2.2 Triangulation through a base currency

If there is no direct or inverse quote available, a quote may be inferred by triangulating through one of the base currencies, which are USD, GBP and EUR. For example, given quotes EURCHF and EURAUD, we can infer a quote `AUDCHF = EURCHF / EURAUD`. If there are quotes on multiple days for a given currency pair, the latest quote will always be taken. In this example, we may have EURCHF quotes for yesterday and today, but only a EURAUD quote for yesterday. The EURCHF quote from today will be combined with the EURAUD quote from yesterday.


We will use the following FX rates:

| FX Pair | Effective At   | Rate       |
| ------- | ------------   | ---------- |
| EURCHF  | Thu 12/09/2024 | 0.9388     |
| EURAUD  | Thu 12/09/2024 | 1.6497     |
| EURCHF  | Fri 13/09/2024 | 0.9415     |

In [20]:
market_data_scope = "fx-inference-triangulate-through-eur"
recipe_code = "fx-inference-triangulate-through-eur"
upsert_recipe(scope=market_data_scope, code=recipe_code)

In [21]:
equity_price = 100
quote_equity = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id=equity_id,
            instrument_id_type="ClientInternal",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=equity_price, unit=equity_ccy),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_equity])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

In [22]:
yesterday = valuation_date + timedelta(days=-1)

# EURCHF Yesterday
spot_eur_chf_yesterday = 0.9388
quote_eur_chf_yesterday = create_fx_quote(
    fx_pair="EUR/CHF", effective_at=yesterday, rate=spot_eur_chf_yesterday
)

# EURCHF Today
spot_eur_chf_today = 0.9415
quote_eur_chf_today = create_fx_quote(
    fx_pair="EUR/CHF", effective_at=valuation_date, rate=spot_eur_chf_today
)

# EURAUD Yesterday
spot_eur_aud_yesterday = 1.6497
quote_eur_aud_yesterday = create_fx_quote(
    fx_pair="EUR/AUD", effective_at=yesterday, rate=spot_eur_aud_yesterday
)

upsert_quotes(
    scope=market_data_scope,
    quotes=[quote_eur_chf_yesterday, quote_eur_chf_today, quote_eur_aud_yesterday],
)

'Quotes successfully loaded into LUSID. 3 quotes upserted.'

With a holding of `10,000` units, worth `100 AUD` each, the present value of the holding is `1,000,000 AUD`. Combining the latest EURCHF and EURAUD quotes gives an inferred quote `AUDCHF = EURCHF / EURAUD = 0.9415 / 1.6497 = 0.57070983`, so based on this inferred rate, the PV of the holding in CHF is `0.57070983 * 1,000,000 = 570,709.83 CHF`. If instead the two rates effective the date before valuation were combined, we would end up with a PV of `0.9388 / 1.6497 * 1,000,000 = 569,073.16 CHF`, which is consistent with directly quoted AUDCHF rates for 12/09/23. On the other hand, the inferred AUDCHF rate composed of rates on different dates of `AUDCHF=0.57070983` is different from the AUDCHF quotes both on 12/09/24 (0.5690) and 13/09/24 (0.5716).

In [23]:
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope, date=valuation_date, recipe_code=recipe_code
)

display(valuation_df)

,Instrument/CoreData/Name,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInPortfolioCcy
0,AnAustralianCompany,"1,000,000.0000",AUD,"570,709.8260"


Finally, we show that rates inferred through one of the base currencies have lower precedence than inverse and direct rates. We have already established that a direct rate takes precedence over an inverse quote, so it remains to show that an inverse quote takes precedence over a quote triangulated through a base currency. Again, the age of the quote is not taken into consideration other than the constraints given by the quote interval.

In [24]:
market_data_scope = "fx-inference-inverse-vs-triangulated"
recipe_code = "fx-inference-inverse-vs-triangulated"
upsert_recipe(scope=market_data_scope, code=recipe_code)

First upsert the equity quote and the three FX quotes above for EURCHF and EURAUD.

In [25]:
upsert_quotes(
    scope=market_data_scope,
    quotes=[
        quote_equity,
        quote_eur_chf_yesterday,
        quote_eur_chf_today,
        quote_eur_aud_yesterday,
    ],
)

'Quotes successfully loaded into LUSID. 4 quotes upserted.'

We will now upsert a CHFAUD quote effective on Wed 11/09/24 and demonstrate that this inverse rate is used in the valuation instead of the rate inferred by triangulating through EUR.

In [26]:
spot_chf_aud = 1 / 0.5671
effective_at = valuation_date + timedelta(days=-2)
quote_chf_aud = create_fx_quote(
    fx_pair="CHF/AUD", effective_at=effective_at, rate=spot_chf_aud
)

upsert_quotes(scope=market_data_scope, quotes=[quote_chf_aud])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

The effective AUDCHF rate based on the inverse quote for `CHFAUD = 1 / 0.5671` is `AUDCHF = 0.5671`, while the effective rate based on triangulating through EUR using the latest quotes is `AUDCHF = EURCHF / EURAUD = 0.9415 / 1.6497 = 0.57070983`. It is evident from the valuation below that the spot rate `AUDCHF = 0.5671` derived from CHFAUD quote is used rather than triangulating through EUR.

In [27]:
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope, date=valuation_date, recipe_code=recipe_code
)

display(valuation_df)

,Instrument/CoreData/Name,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInPortfolioCcy
0,AnAustralianCompany,"1,000,000.0000",AUD,"567,100.0000"


### 2.3 Currency pair quoting conventions

In the example above of triangulating through EUR, all currency pairs were quoted with EUR as the first currency in the pair, however it is not necessary for the middle currency in the triangulation to be used as the base currency in the quotes. For example, in resolving the AUDCHF dependency and triangulating through USD, we can upsert quotes for USDCHF and AUDUSD in line with market quoting conventions, rather than being forced to create a USDAUD quote from available market data. The `Manifest` can be inspected to see which FX rates were used in a valuation.

In fact, any one of the following pairs of quotes is sufficient to infer an AUDCHF spot rate in LUSID:
| Quote Pair     | Expression for AUDCHF |
| -------------- | --------------------- |
| AUDUSD, CHFUSD | AUDUSD / CHFUSD       |
| AUDUSD, USDCHF | AUDUSD * USDCHF       |
| USDAUD, CHFUSD | 1 / (CHFUSD * USDAUD) |
| USDAUD, USDCHF | USDCHF / USDAUD       |


Suppose we have given quotes for AUDUSD, CHFUSD and USDCHF, then the table above does not make it clear which two quotes will be used to infer the AUDCHF quote.

Let `S` denote the set of quotes in the quote store. It is assumed `S` is a subset of `{ AUDUSD, USDAUD, USDCHF, CHFUSD }` since other quotes are irrelevant to this triangulation problem. `S` must contain at least one of `AUDUSD` or `USDAUD` and at least one of `USDCHF` and `CHFUSD`, otherwise an `AUDCHF` quote cannot be inferred. There are 9 subsets satisfying these conditions. There is a (partial) order of preference for which set of quotes is used to form a quote, given by the conditions below:

1. If `S` contains `AUDUSD` and `CHFUSD`, then `AUDCHF = AUDUSD / CHFUSD`.
2. If `S` contains `AUDUSD` and `USDCHF` but does not contain `CHFUSD`, then `AUDCHF = AUDUSD * USDCHF`
3. If `S` contains `USDAUD` and `CHFUSD` but does not contain `AUDUSD`, then `AUDCHF = 1 / (CHFUSD * USDAUD)`
4. If `S` contains only `USDAUD` and `USDCHF`, then `AUDCHF = USDCHF / USDAUD`


We will use the following FX rates:

| FX Pair | Effective At   | Rate       |
| ------- | ------------   | ---------- |
| USDCHF  | Fri 13/09/2024 | 0.8499     |
| AUDUSD  | Fri 13/09/2024 | 0.6727     |

In [28]:
market_data_scope = "fx-inference-triangulate-conventional"
recipe_code = "fx-inference-triangulate-conventional"
upsert_recipe(scope=market_data_scope, code=recipe_code)

In [29]:
equity_price = 100
quote_equity = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id=equity_id,
            instrument_id_type="ClientInternal",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=equity_price, unit=equity_ccy),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_equity])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

In [30]:
spot_usd_chf = 0.8499
quote_usd_chf = create_fx_quote(
    fx_pair="USD/CHF", effective_at=valuation_date, rate=spot_usd_chf
)

spot_aud_usd = 0.6727
quote_aud_usd = create_fx_quote(
    fx_pair="AUD/USD", effective_at=valuation_date, rate=spot_aud_usd
)

upsert_quotes(scope=market_data_scope, quotes=[quote_usd_chf, quote_aud_usd])

'Quotes successfully loaded into LUSID. 2 quotes upserted.'

Given the above quotes `USDCHF = 0.8499` and `AUDUSD = 0.6727`, we can infer a quote `AUDCHF = AUDUSD * USDCHF = 0.6727 * 0.8499 = 0.57172773`. With a holding in AnAustralianCompany worth 1,000,000 AUD, this gives the valuation below by converting at spot: `1,000,000 * 0.57172773 = 571,727.73 CHF`.

In [31]:
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope, date=valuation_date, recipe_code=recipe_code
)

display(valuation_df)

,Instrument/CoreData/Name,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInPortfolioCcy
0,AnAustralianCompany,"1,000,000.0000",AUD,"571,727.7300"


### 2.4 Multi-step triangulation

LUSID does not support triangulation in multiple steps - in absence of a direct or inverse rate, if a rate cannot be inferred by triangulating through a single currency (USD, GBP or EUR), a market data resolution failure will be returned. For this reason, it is recommended to upsert all FX rates against one of the base currencies used in triangulation; which are USD, GBP and EUR. For instance, given quotes for EURXXX for each relevant currency XXX, we can infer all cross rates as `XXXYYY = XXXEUR * EURYYY = EURYYY / EURXXX`.

For example, to price the equity above in the portfolio currency, we require an AUDCHF spot rate. Suppose we are given quotes EURAUD, EURUSD and USDCHF, then in principal we can infer a quote `AUDCHF = EURCHF / EURAUD = (EURUSD * USDCHF) / EURAUD`, but LUSID will not attempt to resolve this quote in two steps. Instead, we need to ensure a full set of quotes is given against one of these base currencies - for instance, if we also upserted a quote `EURCHF = EURUSD * USDCHF`, then LUSID would triangulate through EUR and produce a valuation.

### 2.5 Minor currencies

In LUSID, we maintain a list of the ISO 4217 minor currencies together with the ratio of the major to minor currency - for instance `1 GBP = 100 GBp` and `1 AUD = 100 AUc`. Any minor currency dependencies are transformed to dependencies on the major currency, so we can price an instrument in the minor currency and only need to deal with FX rates involving major currencies (e.g. Not USDGBp).

In [32]:
market_data_scope = "fx-inference-minor-ccys"
recipe_code = "fx-inference-minor-ccys"
upsert_recipe(scope=market_data_scope, code=recipe_code)

In [33]:
# Upsert a quote for the equity: 20 AUD = 2000 AUc
aus_cents = "AUc"
equity_price = 2000

quote_equity = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id=equity_id,
            instrument_id_type="ClientInternal",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=equity_price, unit=aus_cents),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_equity])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

In this example, the price of the AnAustralianCompany is quoted in AUc, so to calculate the PV of the holding in CHF, we need an AUcCHF spot rate. We will now upsert an AUDCHF quote and allow the AUcCHF quote to be inferred using the minor to major currency scale factor.

In [34]:
spot_aud_chf = 0.5671
quote_aud_chf = create_fx_quote(
    fx_pair="AUD/CHF", effective_at=valuation_date, rate=spot_aud_chf
)

upsert_quotes(scope=market_data_scope, quotes=[quote_aud_chf])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

Given the spot rate `AUDCHF = 0.5671`, we calculate `AUcCHF = AUDCHF / 100 = 0.005671`. The holding of 10,000 units in AnAustralianCompany, each worth 2000 AUc, is worth 20,000,000 AUc, which is converted to CHF at the spot `AUcCHF` rate: `PvInPortfolioCcy = 20,000,000 * 0.005671 = 113420.0`.

In [35]:
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope, date=valuation_date, recipe_code=recipe_code
)

display(valuation_df)

,Instrument/CoreData/Name,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInPortfolioCcy
0,AnAustralianCompany,"200,000.0000",AUD,"113,420.0000"
